In [0]:
dbutils.secrets.listScopes()

In [0]:
service_credential = dbutils.secrets.get(scope="azure-key-vault-scope", key="service-credential")
application_id = dbutils.secrets.get(scope="azure-key-vault-scope",key="application-id")
directory_id = dbutils.secrets.get(scope="azure-key-vault-scope", key="directory-id")
storage_account = dbutils.secrets.get(scope="azure-key-vault-scope", key="storage-account")



In [0]:



   # Set Spark configuration to use OAuth 2.0 for authentication
   spark.conf.set("fs.azure.account.auth.type.<storage-account-name>.dfs.core.windows.net", "OAuth")
   spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account-name>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
   spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account-name>.dfs.core.windows.net", "<client-id>")
   spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account-name>.dfs.core.windows.net", "<service_credential>")
   spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account-name>.dfs.core.windows.net", "https://login.microsoftonline.com/<tenant-id>/oauth2/token")

In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", 
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", 
               f"https://login.microsoftonline.com/{directory_id}/oauth2/token")


In [0]:
# Test a basic file access
path = f"abfss://bronze@{storage_account}.dfs.core.windows.net/"
try:
    files = dbutils.fs.ls(path)
    display(files)
except Exception as e:
    print("Error accessing path:", path)
    print(e)


In [0]:
Product_Subcategories_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/Product_Subcategories/")


In [0]:
Territories_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Territories/")


In [0]:
Sales_2017_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Sales_2017/")


In [0]:
Sales_2016_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Sales_2016/")

Sales_2015_df.display()


In [0]:
Sales_2015_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Sales_2015/")


In [0]:
Sales_2015_df.display()

In [0]:
Returns_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Returns/")


In [0]:
Products_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Products/")


In [0]:
Product_Categories_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Product_Categories/")


In [0]:
customers_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Customers/")


In [0]:
calendar_df = spark.read.format("csv") \
    .option("header",True) \
        .option("inferSchema",True) \
            .load(f"abfss://bronze@{storage_account}.dfs.core.windows.net/AdventureWorks_Calendar/")


In [0]:
calendar_df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *



In [0]:


calendar_df = calendar_df.withColumn("Day", dayofmonth(col("Date")))
calendar_df = calendar_df.withColumn("Eng_Month", date_format(col("Date"),"MMMM"))
calendar_df = calendar_df.withColumn("Month", month(col("Date")))

calendar_df = calendar_df.withColumn("Year", year(col("Date")))
calendar_df = calendar_df.drop("Month")
calendar_df.display()



##Merging 3 year Sales data

In [0]:
Sales_2017_df = Sales_2017_df.withColumn("year",lit(2017))
Sales_2016_df = Sales_2016_df.withColumn("year",lit(2016))
Sales_2015_df = Sales_2015_df.withColumn("year",lit(2015))

In [0]:
dfs = [Sales_2016_df,Sales_2017_df,Sales_2015_df]

In [0]:
from functools import *

#####This will work in iteration  by taking 2 df each time and merge the both with next in the next iteration


In [0]:
reduced_sales_df = reduce(lambda x,y:x.unionByName(y),dfs)

In [0]:
customers_df = customers_df.withColumn("FullName",concat_ws((' '),col("prefix"),col("FirstName"),col("LastName")))
customers_df.display()

In [0]:
customers_df.write.format("parquet") \
    .mode("append") \
        .option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/AdventureWorks_Customers/") \
            .save()

In [0]:

calendar_df.write.format("parquet")\
    .mode('append')\
        .option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/AdventureWorks_Calendar/")\
            .save()

In [0]:
Products_df.withColumn("ProductSKU",split(col("ProductSKU"),"-")[0]) \
    .withColumn("ProductName",split(col("ProductName"),",")[0]) \
        .display()

In [0]:
Products_df.write.format("parquet") \
    .mode("append") \
        .option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/Adventureworks_products") \
            .save()

In [0]:
Products_df.display()

In [0]:
Products_df.display()

In [0]:
Returns_df.write.format("parquet") \
    .mode("append") \
        .option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/Adventureworks_returns") \
            .save()

In [0]:
Territories_df.display()

In [0]:
Territories_df.write.format("parquet") \
    .mode("append") \
        .option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/Adventureworks_Territories") \
            .save()

In [0]:
reduced_sales_df.filter(col("year")=='2017').display()

In [0]:
reduced_sales_df = reduced_sales_df.withColumn("StockDate",to_timestamp(col("StockDate")))

In [0]:
reduced_sales_df = reduced_sales_df.withColumn("OrderNumber",regexp_replace(col("OrderNumber"),'S','T'))
reduced_sales_df.display()

In [0]:
reduced_sales_df = reduced_sales_df.withColumn("Total_numberof_items",col("OrderLineItem")*col("OrderQuantity"))

reduced_sales_df.display()

In [0]:
reduced_sales_df.write.format("parquet") \
    .mode("append") \
        .option("Path","abfss://silver@{storage_account}.dfs.core.windows.net/Adventureworks_Sales")

In [0]:
reduced_sales_df.groupBy("OrderDate").agg(count(col("OrderNumber")).alias("Total_orders")).display()

Databricks visualization. Run in Databricks to view.

In [0]:
Returns_df.display()

In [0]:
joined_return_territories_df = Returns_df.join(Territories_df,Returns_df.TerritoryKey == Territories_df.SalesTerritoryKey,'Left_outer')
joined_return_territories_df.display()

In [0]:
regional_return_df = joined_return_territories_df.groupBy("Country").agg(count("ReturnQuantity").alias("Total_sales_return_for region"))

regional_return_df.orderBy("Country").display()

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

In [0]:
Territories_df.display()